In [1]:
from sklearn.datasets import make_classification
import pandas as pd 
import numpy as np 

In [3]:
X, y = make_classification(n_samples=400, n_features=4)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

##### Добавляем регуляризацию

In [6]:
class MyLogReg():
    def __init__(self, n_iter=10, learning_rate=0.1, reg=None, l1_coef=0, l2_coef=0, metric=None, weights=None):
        self.n_iter = n_iter 
        self.learning_rate = learning_rate
        self.__weights = weights
        self.metric = metric
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.__best_score = None

        self.__validate_params()
        
    def __repr__(self):
        return f'MyLogReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'

    def __validate_params(self):
        """Проверяет корректность параметров модели."""
        if self.metric is not None and self.metric not in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
            raise ValueError(f"Invalid metric: {self.metric}, You can only use: 'accuracy', 'precision', 'recall', 'f1' or 'roc_auc' ")
        # Проверям на входные параметры регулиризации 
        if self.reg is not None and self.reg not in ['l1', 'l2', 'elasticnet']:
            raise ValueError("reg must be 'l1', 'l2', or 'elasticnet'")
        # Проверка коэффициентов (должны быть от 0 до 1)
        if self.l1_coef < 0 or self.l1_coef > 1:
            raise ValueError("l1_coef must be in [0, 1]")
        if self.l2_coef < 0 or self.l2_coef > 1:
            raise ValueError("l2_coef must be in [0, 1]")
    
    def __log_training_step(self, iteration: int, y: np.array, proba: np.array, w: np.array):
        """Логирует процесс обучения (итерацию, веса, функцию потерь).
    
        Параметры:
        iteration: int - номер итерации
        X: np.array - матрица признаков
        y: np.array - вектор истинных меток (0 или 1)
        proba: np.array - предсказанные вероятности класса 1
        """
        loss = self.__calculate_loss(y, proba, w)                           # считаем loss 
        metric_value = None                                                 # Инициализируем метрикику 
        if self.metric:                                                     # если она задана 
            y_pred = (proba>0.5).astype(int)                                # вероятности переводим в метки
            metric_value = self.__calculat_metric(y, y_pred if self.metric != 'roc_auc' else proba)       # cчитаем метрику
            
        if iteration == 0:
            if self.metric:
                print(f'start | loss: {loss:0.2f} | {self.metric}: {metric_value:0.2f}')
            else:
                print(f'start | loss: {loss:0.2f}')
        else:
            if self.metric:
                print(f'{iteration} | loss: {loss:0.2f} | {self.metric}: {metric_value:0.2f}')
            else:
                print(f'{iteration} | loss: {loss:0.2f}')

    def __calculate_loss(self, y: np.array, proba: np.array, w: np.array) -> float:
        """
        Вычисляем общий loss (MSLE + регуляризация).
        X: np.array (Матрица признаков)
        y: np.array (Вектор значений)
        w: np.array (Вектор весов)
        """
        eps = 1e-15
        loss = -np.mean(y*np.log(proba+eps) + (1-y)*(np.log(1-proba+eps)))

        if self.reg == 'l1':
            return loss + self.l1_coef * np.sum(np.abs(w))
        elif self.reg == 'l2':
            return loss + self.l2_coef * np.sum(w**2)
        elif self.reg == 'elasticnet':
            return loss + self.l1_coef * np.sum(np.abs(w)) + self.l2_coef * np.sum(w**2)
        else:
            return loss

    def __apply_regularization(self, grad: np.array, w: np.array):
        """Добавляет градиент регуляризации."""
        if self.reg == 'l1':
            return grad + self.l1_coef * np.sign(w)
        elif self.reg == 'l2':
            return grad + 2 * self.l2_coef * w
        elif self.reg == 'elasticnet':
            return grad + self.l1_coef * np.sign(w) + 2 * self.l2_coef * w
        else:
            return grad
    
    def __calculat_metric(self, y_true: np.array, y_pred: np.array):
        """Считает заданные метрики."""
        #  Напишем функцию для подсчета матрицы ошибок
        def confusion_matrix(y_true: np.array, y_pred: np.array):
            # инициализируем нашу матрицу ошибок
            tn = fn = fp = tp = 0 
            for true, pred in zip(y_true, y_pred):
                if pred == 0 and true == 0:          # TN
                    tn += 1 
                elif pred == 0 and true == 1:        # FN
                    fn += 1 
                elif pred == 1 and true == 0:        # FP
                    fp += 1 
                elif pred == 1 and true == 1:        # TP
                    tp += 1 
            return tn, fn, fp, tp
        
        if self.metric == 'roc_auc':
            # Предсказанные вероятности
            y_scores = y_pred
            # Сортируем по вероятностям
            sorted_indices = np.argsort(y_scores)
            sorted_y = np.array(y_true)[sorted_indices]

            # Присваиваем ранги (от 1 до n, как в scipy)
            n = len(y_scores)
            ranks = np.empty(n)
            i = 0
            while i < n:
                j = i
                while j + 1 < n and y_scores[sorted_indices[j]] == y_scores[sorted_indices[j + 1]]:
                    j += 1
                avg_rank = (i + j + 2) / 2  # т.к. ранги начинаются с 1
                for k in range(i, j + 1):
                    ranks[k] = avg_rank
                i = j + 1

            # Считаем сумму рангов положительного класса
            sum_ranks_pos = np.sum(ranks[sorted_y == 1])

            P = np.sum(sorted_y)         # количество положительных
            N = n - P                    # количество отрицательных

            if P == 0 or N == 0:
                return None
            # Вычисляем AUC по формуле Манна-Уитни
            auc = (sum_ranks_pos - P * (P + 1) / 2) / (P * N)
            return auc
        
        tn, fn, fp, tp = confusion_matrix(y_true, y_pred)

        if self.metric == 'accuracy':
            return (tp + tn) / (tp + tn + fp + fn)
        elif self.metric == 'precision':
            return tp / (tp + fp)
        elif self.metric == 'recall':
            return tp / (tp + fn)
        elif self.metric == 'f1':
            pr = tp / (tp + fp)      # precision
            re = tp / (tp + fn)      # recall
            return (2*pr*re) / (pr + re)
        
    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        """
        Метод обучает модель Логистической регресси 
        Входные параметры:
        X: pd.DataFrame
        y: pd.Series
        verbose: bool
        """
        X_copy = X.copy()                                       # копируем матрицу признаков, чтобы не изменить оригинальный 
        X_copy.insert(0, 'base', 1)                             # добавляем столбик для свободного члена, заполним его 1
        X_copy = X_copy.to_numpy()  
        self.__weights = np.ones(X_copy.shape[1])               # создаем вектор весов, заполненный 1

        # Цикл обучения
        for i in range(self.n_iter):
            pred = X_copy.dot(self.__weights)                   # делаем предсказание модели
            proba = 1 / (1 + np.exp(-pred))                     # переводим предсказания в вероятности через функцию сигмоиды
            grad = (1/len(y))*(proba - y).dot(X_copy)           # вычисляем градиент LogLoss
            grad = self.__apply_regularization(grad, self.__weights)   # добавляем регуляризацию
            #self.__weights -= self.learning_rate * grad        # делаем шаг обучения
            if self.metric:                                    # если задана метрика 
                if i == self.n_iter-1:                         # считаем его на n - 1 итерации, чтобы последнне значение записать в best_score
                    marks = (proba>0.5).astype(int)
                    self.__best_score = self.__calculat_metric(y_true=y, y_pred=marks if self.metric!='roc_auc' else proba)

            if verbose and (i % 10 == 0 or i == self.n_iter-1):
                self.__log_training_step(i, y, proba, self.__weights)

            self.__weights -= self.learning_rate * grad          # делаем шаг обучения
            
    def predict_proba(self, X: pd.DataFrame) -> np.array:
        """Вернет вероятностное предсказание модели"""
        X_copy = X.copy()
        X_copy.insert(0, 'base', 1)
        pred = X_copy.dot(self.__weights)
        proba = 1 / (1 + np.exp(-pred))
        return np.array(proba)

    def predict(self, X: pd.DataFrame) -> np.array:
        """Вернет маркерное предсказание модели"""
        proba = self.predict_proba(X)
        return np.array((proba > 0.5).astype(int))

    def get_best_score(self):
        if self.metric is None:
            raise ValueError("Metric was not set during model initialization.")
        return self.__best_score
                
    def get_coef(self) -> np.array:
        return np.array(self.__weights[1:])

In [133]:
model = MyLogReg(reg='elasticnet', l1_coef=0.4, l2_coef=0.45, n_iter=100, metric='accuracy')

In [135]:
X.shape

(400, 4)

In [137]:
model.fit(X, y, verbose=True)

start | loss: 5.06 | accuracy: 0.50
10 | loss: 0.88 | accuracy: 0.87
20 | loss: 0.72 | accuracy: 0.86
30 | loss: 0.72 | accuracy: 0.85
40 | loss: 0.72 | accuracy: 0.84
50 | loss: 0.72 | accuracy: 0.85
60 | loss: 0.72 | accuracy: 0.85
70 | loss: 0.72 | accuracy: 0.85
80 | loss: 0.72 | accuracy: 0.85
90 | loss: 0.72 | accuracy: 0.85
99 | loss: 0.72 | accuracy: 0.86


In [139]:
model.get_coef()

array([ 0.0150875 ,  0.0254203 ,  0.04912361, -0.03112845])

In [115]:
model

MyLogReg class: n_iter=100, learning_rate=0.1